In [2]:
import os, sys
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from joblib import dump

sys.version_info

sys.version_info(major=3, minor=9, micro=11, releaselevel='final', serial=0)

In [8]:
df = pd.read_csv("data_set/train.csv")

columns = ['Elevation', 'Slope', 'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology']

X = df[columns]

X



,Elevation,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology
0,2596,3,258,0
1,2590,2,212,-6
2,2804,9,268,65
3,2785,18,242,118
4,2595,2,153,-1
...,...,...,...,...
15115,2607,23,258,7
15116,2603,19,633,195
15117,2492,25,365,117
15118,2487,28,218,101


In [10]:
scalar = MinMaxScaler()
scalar.fit(X)
X = scalar.transform(X)
types = df["Cover_Type"].values.astype(int)
types

array([5, 5, 2, ..., 3, 3, 3])

In [15]:
# split data into training and test data

X_train, X_test, y_train, y_test = train_test_split(X, types, test_size=0.2, random_state=101)

# train model
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

# Evaluates the trained model using accuracy score
y_predict = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_predict)
accuracy


0.673941798941799

In [17]:
# Save Model
dump(clf, "models/tree_scatter.joblib")

['models/tree_scatter.joblib']

In [ ]:
# houdini node

import pandas as pd
from joblib import load
from sklearn.preprocessing import MinMaxScaler

import hou

node: hou.SopNode = hou.pwd()
geo: hou.Geometry = node.geometry()

columns = ['Elevation', 'Slope', 'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology']

X = pd.DataFrame(columns=columns)

for p in geo.points():
    point_data = [
        p.attribValue("height"),
        p.attribValue("slope"),
        p.attribValue("water"),
        p.attribValue("water"),
    ]
    
    X.loc[p.number()] = point_data


scalar = MinMaxScaler()
scalar.fit(X)
X = scalar.transform(X)
    
clf = load("models/tree_scatter.joblib")

y_predict = clf.predict(X)

for p in geo.points():
    p.setAttribValue("cover_type", int(y_predict[p.number()]))
    


